# Platform Engagement and Growth Analytics

## Stakeholder Questions:
1. What factors are driving or limiting user growth?
2. How do engagement patterns differ by content type or user segment?
3. Which campaigns or acquisition channels bring the most engaged, retained users?
4. How can we maximize ARPU and minimize churn?
5. What content or genres drive the most engagement and repeat sessions?


In [1]:
import duckdb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_rows', None)

con=duckdb.connect()

con.execute("""
CREATE TABLE users_raw AS SELECT * FROM read_csv_auto('data/users.csv');
CREATE TABLE sessions_raw AS SELECT * FROM read_csv_auto('data/sessions.csv');
CREATE TABLE content_raw AS SELECT * FROM read_csv_auto('data/content.csv');
CREATE TABLE marketing_raw AS SELECT * FROM read_csv_auto('data/marketing.csv');
CREATE TABLE revenue_raw AS SELECT * FROM read_csv_auto('data/revenue.csv');
""")

In [2]:
print("\nSample data from each table:")
print("================================")
print("Users Table:")
print(f"\n{con.execute("SELECT * FROM users_raw LIMIT 5").fetchdf()}")
print("================================")
print("Sessions Table:")
print(f"\n{con.execute("SELECT * FROM sessions_raw LIMIT 5").fetchdf()}")
print("================================")
print("Content Table:")
print(f"\n{con.execute("SELECT * FROM content_raw LIMIT 5").fetchdf()}")
print("================================")
print("Marketing Table:")
print(f"\n{con.execute("SELECT * FROM marketing_raw LIMIT 5").fetchdf()}")
print("================================")
print("Revenue Table:")
print(f"\n{con.execute("SELECT * FROM revenue_raw LIMIT 5").fetchdf()}")


Sample data from each table:
Users Table:

  user_id signup_date cancel_date       region plan_type  price_usd  \
0  U00001  2024-05-11         NaT         APAC     BASIC       8.99   
1  U00002  2023-03-28         NaT        LATAM     Basic      10.99   
2  U00003  2023-01-20  2024-05-19  Europe/MENA  Standard      14.99   
3  U00004  2024-07-29         NaT        LatAm  Standard      14.99   
4  U00005  2023-08-02         NaT        LATAM  Standard      13.99   

  acquisition_channel  
0            referral  
1             Organic  
2             organic  
3            Referral  
4            referral  
Sessions Table:

  session_id user_id       date   device  duration_min content_type  \
0   S0000001  U01743 2024-04-22   Tablet         29.65        Movie   
1   S0000002  U01804 2023-01-05   Tablet          8.84       Series   
2   S0000003  U01712 2024-12-04   Tablet          0.00       Shorts   
3   S0000004  U01204 2023-03-03  Desktop         47.17       Shorts   
4   S0000005 

## Data Cleaning

In [3]:
print(con.execute("DESCRIBE users_raw").fetchdf())

print("region values:")
print(con.execute("SELECT region, COUNT(region) FROM users_raw GROUP BY region").fetchdf())

print("plan_type values:")
print(con.execute("SELECT plan_type, COUNT(plan_type) FROM users_raw GROUP BY plan_type").fetchdf())

print("price_usd values:")
print(con.execute("SELECT price_usd, COUNT(price_usd) FROM users_raw GROUP BY price_usd").fetchdf())

print("Acquisition channel values:")
print(con.execute("SELECT acquisition_channel, COUNT(acquisition_channel) FROM users_raw GROUP BY acquisition_channel").fetchdf())

print("\nNull Values:")
print(con.execute("""
SELECT COUNT(*) AS total_rows,
       COUNTIF(region IS NULL) AS null_region,
       COUNTIF(plan_type IS NULL) AS null_plan_type,
       COUNTIF(price_usd IS NULL) AS null_price,
       COUNTIF(user_id IS NULL) AS null_user_id,
       COUNTIF(signup_date IS NULL) AS null_signup_date,
       COUNTIF(acquisition_channel IS NULL) AS null_acquisition_channel,
       COUNTIF(cancel_date IS NULL) AS null_cancel_date
FROM users_raw;
""").fetchdf())

print("Cancel date before signup date:")
print(con.execute("""
SELECT *
FROM users_raw
WHERE cancel_date < signup_date;
""").fetchdf())

           column_name column_type null   key default extra
0              user_id     VARCHAR  YES  None    None  None
1          signup_date        DATE  YES  None    None  None
2          cancel_date        DATE  YES  None    None  None
3               region     VARCHAR  YES  None    None  None
4            plan_type     VARCHAR  YES  None    None  None
5            price_usd      DOUBLE  YES  None    None  None
6  acquisition_channel     VARCHAR  YES  None    None  None
region values:
           region  count(region)
0           LatAm            155
1      N. America            115
2            None              0
3            APAC            181
4     Europe/MENA            174
5              NA            118
6            apac            170
7           laTam            182
8   North America            131
9            EMEA            178
10           emea            158
11          LATAM            176
12   Asia Pacific            145
13  north america            109
plan_type 

In [ ]:
con.execute("""
CREATE OR REPLACE TABLE users_clean AS
WITH ranked AS (
  SELECT
    *,
    ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY signup_date DESC, cancel_date NULLS LAST) AS rn
  FROM users_raw
)
SELECT
  user_id,
  CAST(signup_date AS DATE) AS signup_date,
  CASE 
    WHEN TRY_CAST(cancel_date AS DATE) IS NULL THEN NULL
    WHEN TRY_CAST(cancel_date AS DATE) < TRY_CAST(signup_date AS DATE) THEN NULL  -- fix bad ordering
    ELSE CAST(cancel_date AS DATE)
  END AS cancel_date,
  CASE
    WHEN LOWER(region) IN ('na','n. america','north america') THEN 'North America'
    WHEN LOWER(region) LIKE '%lat%' THEN 'LATAM'
    WHEN LOWER(region) LIKE '%emea%' OR LOWER(region) LIKE '%europe%' THEN 'EMEA'
    WHEN LOWER(region) LIKE '%apac%' OR LOWER(region) LIKE '%asia%' THEN 'APAC'
    ELSE COALESCE(NULLIF(TRIM(region),''), 'Unknown')
  END AS region,
  CASE
    WHEN LOWER(plan_type) LIKE 'basic%' THEN 'Basic'
    WHEN LOWER(plan_type) LIKE 'standard%' THEN 'Standard'
    WHEN LOWER(plan_type) LIKE 'premium%' THEN 'Premium'
    ELSE 'Unknown'
  END AS plan_type,
  TRY_CAST(price_usd AS DOUBLE) AS price_usd,
  CASE
    WHEN LOWER(acquisition_channel) LIKE '%organic%' THEN 'Organic'
    WHEN LOWER(acquisition_channel) LIKE '%social%' THEN 'Social'
    
    ELSE 'Other'
  END AS acquisition_channel
FROM ranked
WHERE rn = 1
  AND user_id IS NOT NULL
;
""")

In [5]:
print(con.execute("DESCRIBE users_raw").fetchdf())

print("region values:")
print(con.execute("SELECT region, COUNT(region) FROM users_raw GROUP BY region").fetchdf())

print("plan_type values:")
print(con.execute("SELECT plan_type, COUNT(plan_type) FROM users_raw GROUP BY plan_type").fetchdf())

print("price_usd values:")
print(con.execute("SELECT price_usd, COUNT(price_usd) FROM users_raw GROUP BY price_usd").fetchdf())

print("Acquisition channel values:")
print(con.execute("SELECT acquisition_channel, COUNT(acquisition_channel) FROM users_raw GROUP BY acquisition_channel").fetchdf())

print("\nNull Values:")
print(con.execute("""
SELECT COUNT(*) AS total_rows,
       COUNTIF(region IS NULL) AS null_region,
       COUNTIF(plan_type IS NULL) AS null_plan_type,
       COUNTIF(price_usd IS NULL) AS null_price,
       COUNTIF(user_id IS NULL) AS null_user_id,
       COUNTIF(signup_date IS NULL) AS null_signup_date,
       COUNTIF(acquisition_channel IS NULL) AS null_acquisition_channel,
       COUNTIF(cancel_date IS NULL) AS null_cancel_date
FROM users_raw;
""").fetchdf())

print("Cancel date before signup date:")
print(con.execute("""
SELECT *
FROM users_raw
WHERE cancel_date < signup_date;
""").fetchdf())

           column_name column_type null   key default extra
0              user_id     VARCHAR  YES  None    None  None
1          signup_date        DATE  YES  None    None  None
2          cancel_date        DATE  YES  None    None  None
3               region     VARCHAR  YES  None    None  None
4            plan_type     VARCHAR  YES  None    None  None
5            price_usd      DOUBLE  YES  None    None  None
6  acquisition_channel     VARCHAR  YES  None    None  None
region values:
           region  count(region)
0            APAC            181
1     Europe/MENA            174
2              NA            118
3            apac            170
4           LatAm            155
5      N. America            115
6            None              0
7           LATAM            176
8    Asia Pacific            145
9   north america            109
10          laTam            182
11  North America            131
12           EMEA            178
13           emea            158
plan_type 

In [6]:
print(con.execute("DESCRIBE sessions_raw").fetchdf())



            column_name column_type null   key default extra
0            session_id     VARCHAR  YES  None    None  None
1               user_id     VARCHAR  YES  None    None  None
2                  date        DATE  YES  None    None  None
3                device     VARCHAR  YES  None    None  None
4          duration_min      DOUBLE  YES  None    None  None
5          content_type     VARCHAR  YES  None    None  None
6            content_id     VARCHAR  YES  None    None  None
7  watch_completion_pct      DOUBLE  YES  None    None  None


## Metrics

| KPI                                       | Derived From         | Formula / Logic                               | Universally Applicable Insight    |
| ----------------------------------------- | -------------------- | --------------------------------------------- | --------------------------------- |
| **DAU / MAU**                             | Sessions             | Count distinct users per day/month            | Engagement intensity              |
| **Engagement Rate**                       | Sessions             | Sessions per active user per day              | How “sticky” product is           |
| **Average Watch Time / Session Duration** | Sessions             | Avg(duration_min)                             | Session quality                   |
| **Completion Rate**                       | Sessions             | Avg(watch_completion_pct)                     | Content relevance or satisfaction |
| **Retention Rate**                        | Users                | (Returning users at t+30 / users active at t) | User loyalty                      |
| **Churn Rate**                            | Users                | 1 – retention                                 | Attrition analysis                |
| **Subscriber Growth Rate**                | Users                | (New subs – churned subs) / prior subs        | Growth health                     |
| **ARPU (Average Revenue per User)**       | Revenue              | Total revenue / active users                  | Monetization efficiency           |
| **Ad CPM**                                | Marketing + Revenue  | ad_revenue_usd / (impressions/1000)           | Ad efficiency                     |
| **Engagement per $ Spent**                | Marketing + Sessions | active_users / spend_usd                      | Marketing ROI                     |
